In [0]:
from pyspark.sql.functions import udf

from pyspark.sql.types import FloatType, ArrayType, LongType, StringType

In [0]:
## TODO Recording for cell below

# Please look in commands.sh for creating table through UI
# Please expand the data frame and show

In [0]:
superstore_df = spark.sql('SELECT * FROM superstore_data')

superstore_df.display()

In [0]:
def cost(sales, profit):
  if profit < 0:
    return (sales + profit)
  else:
    return (sales - profit)

In [0]:
def cost(sales, profit):
  return (int(sales) - int(profit))

In [0]:
#its possible to mix and match and run sql statements with dataframe statements like the two commands below.
superstore_subset_df = spark.sql('select Product_Name, Sales, Profit from superstore_data where Sales IS NOT NULL')

superstore_subset_df.display()


In [0]:
loan_data_subset_df.filter('Type = "Car"')\
                   .display()

In [0]:
compute_cost_udf = udf(cost, FloatType())

superstore_subset_df.withColumn('Cost', compute_cost_udf('Sales', 'Profit'))\
             .select('Product_Name', 'Sales', 'Profit', 'Cost')\
             .display()

In [0]:
superstore_with_cost_df = superstore_df.select('Product_Name', 'Sales', 'Profit', 
                                               compute_cost_udf('sales', 'profit').alias('Cost'))

superstore_with_cost_df.display()

In [0]:
from typing import Optional

@udf(returnType = FloatType())
def profit_percent_udf(profit, cost) -> Optional[float]:
  if cost:
    return (profit / cost) * 100
  return None

In [0]:
superstore_with_cost_df.select('Product Name', 'Sales', 'Profit', 'Cost', 
                               profit_percent_udf('profit', 'cost').alias('Profit Percent')).display()

In [0]:
@udf(returnType = StringType())
def categorize_udf(category, sub_category):
  
  if category and sub_category:
    return '{0} ({1})'.format(category, sub_category)
  elif category:
    return category
  elif sub_category:
    return sub_category
  
  return None

In [0]:
superstore_df.select('Product Name',
                     categorize_udf('Category', 'Sub-Category').alias('Category')).display()

In [0]:
# first we have to register our dataframe as table 
# then we have to register our function with spark udf, so that we can make use of sql queries

In [0]:
superstore_with_cost_df.display()

In [0]:
superstore_with_cost_df.createOrReplaceTempView('superstore_data_with_cost')

In [0]:
spark.udf.register('profit_percent', profit_percent_udf)

In [0]:
spark.sql("""select `product name`, profit, cost, profit_percent(profit, cost) as profit_percent 
             from superstore_data_with_cost""").display()

In [0]:
spark.udf.register('categorize', categorize_udf)

In [0]:
spark.sql("""select `product name`, categorize(category, `sub-category`) as category 
             from superstore_data""").display()